In [ ]:
import json
import numpy as np
from utils.utilities import remove_ids, eval_tagging_scores, calculate_accuracy_varying_lengths
import os
import copy
import os
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.2f')
from utils.SemanticMatch import T5ForSoftMatching, BertForSoftMatching, T5V1_1ForSoftMatching
from utils.misc import remove_entity_index,check_alignment,cal_triplet_acc_score_vrdFormer
from utils.eval_func import eval_pred_data
import os
class NumpyFloatValuesEncoder(json.JSONDecoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return json.JSONDecoder.default(self, obj)

In [ ]:
from utils.utilities import get_AG_annotations_framewise, AG_Objects,AG_relationsCombined

dataset_name = "ActionGnome"

splits = ["test"]
# VIDEO_ROOT_PATH = "/groups/sernam/datasets/ActionGenome/Charades_v1_480"
# OUTPUT_JSON_DIR = "/home/jbhol/dso/gits/ActionGenome/inference/AG_llava_annotations_v5_3"
AG_ANNOTATIONS_DIR = "D:\\Datasets\\ActionGenome\\data"
AG_ANNOTATIONS_VID_DIR = "D:\\Datasets\\ActionGenome\\Charades_v1_480"
CHUNK_N = 1000 # Q&A will be chunked into CHUNK_N parts
AG_Annotations,dataset_meta,video_frame_data = get_AG_annotations_framewise(AG_ANNOTATIONS_DIR=AG_ANNOTATIONS_DIR, 
                                                                            subset=splits[0])

DATASET_DATA = {
    "dataset_subjects": AG_Objects,
    "dataset_objects": AG_Objects,
    "dataset_predicates": AG_relationsCombined
}

In [ ]:
# with open(".\\inference_outputs_onevision\\vidvrd_v13_indexedlist3\\vidvrd_inference_val_vidvrd_v13_indexedlist3.json") as f:
#     pred_data = eval(json.loads(f.read()))
# PredJsons = [
#     "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\[test][onevision]_AG_annotations_v5_3_p00_e01_all_mm_tune\\ActionGnome_inference_val.json"
# ]
# with open(PredJsons[0]) as f:
#     pred_data = json.loads(f.read())
# cleaned_outputs = {}
# for video_id, video_data in pred_data.items():
#     if video_id not in cleaned_outputs.keys():
#         cleaned_outputs[video_id] = {}

#     for block_id,block_data in video_data.items():
#         if block_id not in cleaned_outputs[video_id]:
#             cleaned_outputs[video_id][block_id] = {}

#         cleaned_outputs[video_id][block_id] = {
#             "frames": block_data["frames"],
#             "GT_triplets": block_data["GT_triplets"],
#             "triplets": block_data["triplets"],
#             # "triplets_bb": block_data["triplets_bb"],
#             # "GT_triplets_BB": block_data["GT_triplets_BB"]
#         }

In [ ]:
def consolidate_results(jsons_list):
    cleaned_outputs = {}
    for rawData in jsons_list:
        triplet_key = "cleaned_output"
        print(rawData)
        with open(rawData) as f:
            raw_data = json.loads(f.read())
            for raw_data_item in [raw_data]:
                for video_id, video_data in raw_data_item.items():
                    if video_id not in cleaned_outputs.keys():
                        cleaned_outputs[video_id] = {}
                    for block_id,block_data in video_data.items():
                        
                        if "cleaned_output" not in block_data.keys() and "triplets" not in block_data.keys():
                            continue

                        if "cleaned_output" not in block_data.keys():
                            triplet_key = "triplets"
                        else:
                            block_data["triplets"] = copy.deepcopy(block_data["cleaned_output"])
                            del block_data["cleaned_output"]
                            
                        # if "triplets" not in block_data.keys():
                        #     triplet_key = "cleaned_output"

                        if "triplets" in block_data.keys():
                            if block_id not in cleaned_outputs[video_id]:
                                cleaned_outputs[video_id][block_id] = {}

                            cleaned_outputs[video_id][block_id] = {
                                "frames": block_data["frames"],
                                "GT_triplets": block_data["GT_triplets"],
                                "triplets": block_data["triplets"],
                                # "triplets": block_data["cleaned_output"],
                            }
    return cleaned_outputs

In [ ]:
Jsons_ToCombine = [
    # # p00
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\FFT\\[test][onevision]_AG_annotations_v5_3_p0_e01\\results_raw_response.json",
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\FFT\\[test][onevision]_AG_annotations_v5_3_p0_e01_rem\\ActionGnomestart_idx_1399_inference_val_raw_response.json"

    # #p01
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\FFT\\[test][onevision]_AG_annotations_v5_3_p01_e01_fulltune\\ActionGnome_inference_val_raw_response.json",
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\FFT\\[test][onevision]_AG_annotations_v5_3_p01_e01_fulltune\\ActionGnomestart_idx_955_inference_val_raw_response.json"
    
    # ## p02
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\FFT\\[test][onevision]FFT_AG_annotations_v5_3_p02_e01\ActionGnome_inference_val_raw_response.json"

    # ## P06
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\FFT\\[test][onevision]_AG_annotations_v5_3_p06_e01_fulltune\\ActionGnome_inference_val_raw_response.json",
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\FFT\\[test][onevision]_AG_annotations_v5_3_p06_e01_fulltune\\ActionGnomestart_idx_760_inference_val_raw_response.json"

    # ## p023
    # "inference_outputs_onevision\ConsolidatedResults\llava_ov\AG\FFT\[test][onevision]_AG_annotations_v5_3_p023_e01_fulltune\ActionGnome_inference_val_raw_response.json",
    # "inference_outputs_onevision\ConsolidatedResults\llava_ov\AG\FFT\[test][onevision]_AG_annotations_v5_3_p023_e01_1111_fulltune\ActionGnomestart_idx_1111_inference_val_raw_response.json",
    # "inference_outputs_onevision\ConsolidatedResults\llava_ov\AG\FFT\[test][onevision]_AG_annotations_v5_3_p023_e01_1673_fulltune\ActionGnomestart_idx_1673_inference_val_raw_response.json",
    # "inference_outputs_onevision\ConsolidatedResults\llava_ov\AG\FFT\[test][onevision]_AG_annotations_v5_3_p023_e01_1673_fulltune_run2\ActionGnome_inference_val_raw_response.json"

    # ## LLAVA Qwen2
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_vid_qwen2\\[test][llavavid_qwen2]_AG_annotations_v5_3_p06_e01_lora\\ActionGnome_inference_val_raw_response_json-fixer.json",
    # "inference_outputs_onevision\\ConsolidatedResults\\llava_vid_qwen2\\[test][llavavid_qwen2]_AG_annotations_v5_3_p06_e01_lora_1426\\ActionGnomestart_idx_1426_inference_val_raw_response_json-fixer.json"

    # # P23
    # "inference_outputs_onevision\ConsolidatedResults\llava_vid_qwen2\[test][llavavid_qwen2]_AG_annotations_v5_3_p023_e01_lora\ActionGnome_inference_val_raw_response.json",
    # "inference_outputs_onevision\ConsolidatedResults\llava_vid_qwen2\[test][llavavid_qwen2]_AG_annotations_v5_3_p023_e01_lora_704\ActionGnomestart_idx_704_inference_val_raw_response.json",



]
cleaned_outputs = consolidate_results(Jsons_ToCombine)

In [ ]:
len(cleaned_outputs.keys())

In [ ]:
LORA_AG_RawDataToClean = [
    "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\[test][onevision]_AG_annotations_v5_3_p02_e01_all_mm_tune\\ActionGnome_inference_val_raw_response.json",
    "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\[test][onevision]_AG_annotations_v5_3_p06_e01\\ActionGnome_inference_val_raw_response.json",
    "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\\AG\\[test][onevision]_AG_annotations_v5_3_p023_e01_all_mm_tune\\ActionGnome_inference_val_raw_response.json",
    "inference_outputs_onevision\\ConsolidatedResults\\llava_ov\AG\\[test][onevision]_AG_annotations_v5_3_p01_e01\\results_raw_response.json",
    "inference_outputs_onevision\\AG\\[test][onevision]_AG_annotations_v5_3_p00_e01_all_mm_tune\\ActionGnome_inference_val_json-fixer.json"
]

cleaned_outputs = {}
for rawData in [LORA_AG_RawDataToClean[4]]:
    print(rawData)
    with open(rawData) as f:
        raw_data = json.loads(f.read())
        for raw_data_item in [raw_data]:
            for video_id, video_data in raw_data_item.items():
                if video_id not in cleaned_outputs.keys():
                    cleaned_outputs[video_id] = {}
                for block_id,block_data in video_data.items():
                    if block_id not in cleaned_outputs[video_id]:
                        cleaned_outputs[video_id][block_id] = {}
                    cleaned_outputs[video_id][block_id] = {
                        "frames": block_data["frames"],
                        "GT_triplets": block_data["GT_triplets"],
                        "triplets": block_data["triplets"],
                        # "triplets": block_data["cleaned_output"],
                    }
# total_keys = 0
# for raw_data_item in [raw_data,raw_data2]:
#     print(len(raw_data_item.keys()))
#     total_keys +=len(raw_data_item.keys())
# print("total", total_keys)
# len(raw_data.keys()), len(raw_data2.keys()), len(raw_data.keys())+len(raw_data2.keys())

In [ ]:
# from utils.utilities import pre_clean_prediction_data_v18

In [ ]:
class PredConfig:
    topk = 1
pred_config = PredConfig()
# before_alignment, all_triplets_pairs, [new_subjects,new_predicates,new_objects] = eval_pred_data(data=cleaned_outputs,**DATASET_DATA, PredConfig=pred_config)

for R in [1,10,20,50,100]:
    pred_config.topk = R
    before_alignment, all_triplets_pairs, [new_subjects,new_predicates,new_objects] = eval_pred_data(data=cleaned_outputs,**DATASET_DATA, PredConfig=pred_config)

    print(f"TOP-{R}",before_alignment["VRDFormer_Logic"]["triplet"])

In [ ]:
before_alignment

In [ ]:
before_alignment

In [ ]:
before_alignment

In [ ]:
# inference_output_dir = "AG_llava_annotations/inference_output/[test][onevision]_AG_annotations_v5_3_p0_e01_combined"
# dataset_name = "ActionGnome"
# os.makedirs(inference_output_dir,exist_ok=True)
# try:
#     before_alignment["dataset_meta"] ={
#         "dataset_triplets_existing": DATASET_DATA,
#         "dataset_triplets_new": {
#             "dataset_subjects": new_subjects,
#             "dataset_objects": new_objects,
#             "dataset_predicates": new_predicates
#         }
#     }
# except Exception as e:
#     pass

# try:
#     outputfile = f"{inference_output_dir}/{dataset_name}_inference_val.json"
#     # outputfile = f"{inference_output_dir}/results.json"
#     with open(outputfile, "w") as f:
#         json.dump(cleaned_outputs,f, indent=4)
# except Exception as e:
#     print(f"error saving file: {e}")

# try:
#     outputfile = f"{inference_output_dir}/results_eval_data.json"
#     with open(outputfile, "w") as f:
#         json.dump(before_alignment,f, indent=4)
# except Exception as e:
#     print(f"error saving file: {e}")

In [ ]:
new_objects

In [ ]:
list(all_triplets_pairs.keys())[0]

In [ ]:
all_triplets_pairs["00607.mp4"]["Block0"]

In [ ]:


cleaned_outputs.keys()

In [ ]:
import copy

In [ ]:
VideoId = list(cleaned_outputs.keys())[0]
print(VideoId)
Blockid = list(cleaned_outputs[VideoId].keys())[0]
print(Blockid)


frames = copy.deepcopy(cleaned_outputs[VideoId][Blockid]["frames"])
GT_triplets = copy.deepcopy(cleaned_outputs[VideoId][Blockid]["GT_triplets"])
triplets = copy.deepcopy(cleaned_outputs[VideoId][Blockid]["triplets"])
triplets_bb = copy.deepcopy(cleaned_outputs[VideoId][Blockid]["triplets_bb"])
GT_triplets_BB = copy.deepcopy(cleaned_outputs[VideoId][Blockid]["GT_triplets_BB"])

In [ ]:
triplets_bb

In [ ]:
frame_zero_idx = 3
current_frame_indexes = frames[frame_zero_idx]

current_GT_triplets = GT_triplets[frame_zero_idx]
print(current_GT_triplets)
current_GT_triplets_bb = GT_triplets_BB[frame_zero_idx]
print(current_GT_triplets_bb)

# current_pred_triplets = triplets[frame_zero_idx]
# print(current_pred_triplets)

# current_pred_triplets_bb = triplets_bb[frame_zero_idx]
# print(current_pred_triplets_bb)

In [ ]:
current_frame_indexes

In [ ]:
import cv2
import matplotlib.pyplot as plt

font                   = cv2.FONT_HERSHEY_PLAIN
bottomLeftCornerOfText = (10,500)
fontScale              = 1.0
fontColor_GT           = (0,150,255)
fontColor_PRED         = (0,255,0)
thickness              = 1
lineType               = 1

In [ ]:
video_id = VideoId
capture = cv2.VideoCapture(os.path.join(AG_ANNOTATIONS_VID_DIR,f"{VideoId}"))

# k=capture.isOpened()
# if k==False:
#    capture.open(os.path.exists(os.path.join(imagenet_vidvrd_video_path,f"{video_id}.mp4")))

capture.set(cv2.CAP_PROP_POS_FRAMES,frames[frame_zero_idx])
ret, frame = capture.read()
print(ret)
if ret:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

In [ ]:
h,w,c = frame.shape

In [ ]:
plt.imshow(frame)

In [ ]:
from utils.utilities import unnormbb

In [ ]:
triplets_bb["frame-3"][0]["dish"]

In [ ]:
pred_box = unnormbb(pred_box=copy.deepcopy(triplets_bb["frame-3"][0]["dish"]),mask=frame)

In [ ]:
pred_box

In [ ]:
np.array([240,360]).shape

In [ ]:
# GTboxes = cv2.rectangle(frame, (GT_box[0], GT_box[1]), (GT_box[2], GT_box[3]), (0,0,255), 2)
PredBoxes_img = cv2.rectangle(frame, (pred_box[0], pred_box[1]), (pred_box[2], pred_box[3]), (255,0,0), 2)

In [ ]:
plt.imshow(frame)

In [ ]:
# import os
# os.makedirs("AG_llava_annotations\\inference_output\\Qualitative\\lang_adapter",exist_ok=True)

In [ ]:
# all_triplets_pairs["00607.mp4"]["Block0"]
for video_id, video_data in all_triplets_pairs.items():


    capture = cv2.VideoCapture(os.path.join(AG_ANNOTATIONS_VID_DIR,f"{video_id}"))

    video_save_root = f"AG_llava_annotations\\inference_output\\Qualitative\\lang_adapter\\{video_id}"
    os.makedirs(video_save_root,exist_ok=True)

    for block_id, block_data in video_data.items():

        video_frame_ids = eval(block_data["frames"])

        for frame_zero_idx, frame_data in block_data.items():
            if frame_zero_idx=="frames":
                continue

                
            actual_frame_idx = video_frame_ids[frame_zero_idx]

            GT_triplets = frame_data["gt_triplets"]
            pred_triplets = frame_data["pred_triplets"]
            
            capture.set(cv2.CAP_PROP_POS_FRAMES,int(actual_frame_idx))
            ret, frame = capture.read()

            sg_mask = np.zeros_like(frame)
            text_loc = [10,50]
            for f_sg in GT_triplets:
                sub,pred,obj = f_sg
                cv2.putText(sg_mask, f"{sub}:{pred}:{obj}", (text_loc[0], text_loc[1]), font, fontScale,fontColor_GT,thickness,lineType)
                text_loc[1] +=30

            sg_mask_pred = np.zeros_like(frame)
            text_loc = [10,50]
            for f_sg in pred_triplets:
                sub,pred,obj = f_sg
                cv2.putText(sg_mask_pred, f"{sub}:{pred}:{obj}", (text_loc[0], text_loc[1]), font, fontScale,fontColor_PRED,thickness,lineType)
                text_loc[1] +=30

            stacked_image = np.hstack([frame,sg_mask,sg_mask_pred])
            cv2.imwrite(f"{video_save_root}\\{actual_frame_idx}.jpg", stacked_image)
    capture.release()

In [ ]:
plt.imshow(frame)

In [ ]:
def printScore(data):
    score_str = ""
    for key in ["triplet", "subject", "predicate", "object"]:
        precision_before = data["VRDFormer_Logic"][key]["Precision@1"]
        recall_before = data["VRDFormer_Logic"][key]["Recall@1"]
        score_str += f"[{key}: {precision_before:.3f} {recall_before:.3f}]"
    print(score_str)

for model_cls in [T5ForSoftMatching]:
    model = model_cls()
    print("using ",model.__class__)
    model.register_new_gallery(gallery_name="predicates",gallery_data=AG_relationsCombined)
    model.register_new_gallery(gallery_name="objects",gallery_data=AG_Objects)

    ALIGNMENT_CONFIDENCE_THR = 0.80 # if less dont replace the entity
    ALIGNMENT_MODEL = model

    new_predicates_aligned = ALIGNMENT_MODEL.align_entities(data_to_align=new_predicates,gallery_name="predicates",confidence_thr=ALIGNMENT_CONFIDENCE_THR)
    new_objects_aligned = ALIGNMENT_MODEL.align_entities(data_to_align=new_objects,gallery_name="objects",confidence_thr=ALIGNMENT_CONFIDENCE_THR)
    new_subjects_aligned = ALIGNMENT_MODEL.align_entities(data_to_align=new_subjects,gallery_name="objects",confidence_thr=ALIGNMENT_CONFIDENCE_THR)

    ALIGNMENT_DATA = {
        'check_alinged': True,
        'alinged_subjects': new_subjects_aligned,
        'alinged_objects': new_objects_aligned,
        'alinged_predicates': new_predicates_aligned
    }

    after_alignment, all_triplets_pairs_after_alignment, [new_subjects_,new_predicates_,new_objects_] = eval_pred_data(
        pred_data,
        **DATASET_DATA,
        **ALIGNMENT_DATA)

    print() 
    printScore(before_alignment)
    print(f"### after alignment with {model.__class__}")
    printScore(after_alignment)
    print(cal_triplet_acc_score_vrdFormer(before_alignment), cal_triplet_acc_score_vrdFormer(after_alignment))
    print()
    del model

In [ ]:
before_alignment,after_alignment

In [ ]:
new_subj_str = ""
for subj in new_subjects:
    new_subj_str += f" '{subj}',"
print(new_subj_str)

In [ ]:
new_obj_str = ""
for obj in new_objects:
    new_obj_str += f" '{obj}',"
print(new_obj_str)

In [ ]:
new_predicates_str = ""
for pred in new_predicates:
    new_predicates_str += f" '{pred}',"
print(new_predicates_str)